# ADF test
Augmented Dicky Fuller tests

In [ ]:

import numpy as np
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.tsatools import lagmat, add_trend
from statsmodels.tsa.adfvalues import mackinnonp

def adf(ts):
    """
    Augmented Dickey-Fuller unit root test
    """
    # make sure we are working with an array, convert if necessary
    ts = np.asarray(ts)
    
    # Get the dimension of the array
    nobs = ts.shape[0]
    
    # We use 1 as maximum lag in our calculations
    maxlag = 1
    
    # Calculate the discrete difference
    tsdiff = np.diff(ts)
    
    # Create a 2d array of lags, trim invalid observations on both sides
    tsdall = lagmat(tsdiff[:, None], maxlag, trim='both', original='in')
    # Get dimension of the array
    nobs = tsdall.shape[0] 
    
    # replace 0 xdiff with level of x
    tsdall[:, 0] = ts[-nobs - 1:-1]  
    tsdshort = tsdiff[-nobs:]
    
    # Calculate the linear regression using an ordinary least squares model    
    results = OLS(tsdshort, add_trend(tsdall[:, :maxlag + 1], 'c')).fit()
    adfstat = results.tvalues[0]
    
    # Get approx p-value from a precomputed table (from stattools)
    pvalue = mackinnonp(adfstat, 'c', N=1)
    return pvalue

# Herst exponent
to calculate the speed of divergence

In [ ]:
# https://medium.com/bluekiri/simple-stationarity-tests-on-time-series-ad227e2e6d48
def herst(ts):
    ts = np.array(ts)
    
    # Helper
    lagvec = []
    tau = []
    #range of lag variables
    lags = range(2, 100)
    for lag in lags:
        # lag difference
        pdiff = np.subtract(ts[lag:], ts[:-lag])
        
        lagvec.append(lag)
        # variance of the difference vector
        tau.append(np.square(np.std(pdiff)))
        
    #Linear fit to double-log graph
    m = np.polyfit(np.log10(np.asarray(lagvec)),
                   np.log10(np.asarray(tau).clip(min=0.0000000001)),
                   1)
     
    # return the calculated hurst exponent
    return m[0]*2.0

# Variance Ratio test

$$ \frac{Var(z(t) -z(t-\tau)}{\tau Var(z(t) - z(t-1))}$$

In [ ]:
def variance_ratio(ts, lag=2):
    ts = np.asarray(ts)
    n = len(ts)
    mean_lag1 = sum(ts[1:n]-ts[:n-1])/n
    m = (n-lag+1)*(1-lag/n)
    var_diff_1 = sum(np.square(ts[1:n]-ts[:n-1] - mean_lag1))/(n-1)
    
    mean_lag = lag*mean_lag1
    t = sum( np.square(ts[lag:n] - ts[:n-lag] - lag*mean_lag))/m
    
    return t/(lag*var_diff_1)

# Half life of mean reversion
$$ \Delta y(t) = \lambda y(t-1) + \mu + \beta * t + \alpha_1 \Delta y(t-1) + ... + \epsilon_t
$$
a simplified version:
$$ $$